In [2]:
import pandas as pd
from sodapy import Socrata
import matplotlib.pyplot as plt

In [3]:
#initial test pull, used docs: https://dev.socrata.com/foundry/data.pa.gov/dc5b-gebx

client = Socrata("data.pa.gov", None)
results = client.get("dc5b-gebx", limit=None)
results_df = pd.DataFrame.from_records(results)

In [5]:
results_df.shape

(1000, 215)